In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error

def rmsle(y_true, y_pred):
    #y_pred = np.maximum(0, y_pred)  # RMSLE requires non-negative predictions
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

def parse_datetime(x):
    for fmt in ("%Y-%m-%d %H:%M:%S", "%d-%m-%Y %H:%M"):
        try:
            return pd.to_datetime(x, format=fmt)
        except:
            pass
    return pd.to_datetime(x)   # fallback

def display_all(df):
    for col in df.columns:
        #print("="*40)
        #print(f"Column Name : {col}, Distinct values: {df[col].nunique()}, Values: {df[col].unique()}")
        print(f"Column Name : {col}, Distinct values: {df[col].nunique()}")
        #print(df[col].value_counts())

def display_data(df, selectd):
    for col in selectd:
        #print("="*40)
        #print(f"Column Name : {col}, Distinct values: {df[col].nunique()}, Values: {df[col].unique()}")
        print(f"Column Name : {col}, Distinct values: {df[col].nunique()}")
        #print(df[col].value_counts())

In [2]:
df = pd.read_csv("bike_train.csv")
display_all(df)

Column Name : datetime, Distinct values: 10450
Column Name : season, Distinct values: 4
Column Name : holiday, Distinct values: 2
Column Name : workingday, Distinct values: 2
Column Name : weather, Distinct values: 4
Column Name : temp, Distinct values: 1801
Column Name : atemp, Distinct values: 1815
Column Name : humidity, Distinct values: 92
Column Name : windspeed, Distinct values: 1676
Column Name : casual, Distinct values: 296
Column Name : registered, Distinct values: 716
Column Name : count, Distinct values: 817


In [3]:
df.head(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2012-07-15 7:00:00,3,0,0,1,28.70000,33.335000,79,6.003200,17,30,47
1,2012-08-14 15:00:00,3,0,1,1,33.62000,37.880000,46,15.001300,84,199,283
2,2011-02-06 6:00:00,1,0,0,1,10.66000,12.880000,60,15.001300,0,1,1
3,2012-05-06 17:00:02,2,0,0,2,26.42506,30.566166,61,9.512288,198,330,531
4,2012-01-09 2:00:00,1,0,1,1,9.84000,12.120000,56,8.998100,2,3,5


In [4]:
print(df.isna().sum())

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64


In [5]:
print(df.dtypes)

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object


In [6]:
def add_features(df):

    df["datetime"] = df["datetime"].apply(parse_datetime)
    
    # Extract useful parts (but NOT using hour/year raw later)
    df["hour"] = df["datetime"].dt.hour
    df["weekday"] = df["datetime"].dt.weekday
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year
    
    # ----------------------------
    # Cyclical Hour Encoding
    # ----------------------------
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    return df

df = add_features(df)

In [7]:
selected = ["datetime", "hour", "weekday", "day", "month", "year"]
print(df[selected])

                 datetime  hour  weekday  day  month  year
0     2012-07-15 07:00:00     7        6   15      7  2012
1     2012-08-14 15:00:00    15        1   14      8  2012
2     2011-02-06 06:00:00     6        6    6      2  2011
3     2012-05-06 17:00:02    17        6    6      5  2012
4     2012-01-09 02:00:00     2        0    9      1  2012
...                   ...   ...      ...  ...    ...   ...
10445 2012-10-14 13:00:00    13        6   14     10  2012
10446 2011-03-12 22:00:00    22        5   12      3  2011
10447 2011-04-07 06:00:00     6        3    7      4  2011
10448 2012-08-19 09:00:00     9        6   19      8  2012
10449 2012-01-15 00:00:00     0        6   15      1  2012

[10450 rows x 6 columns]


In [8]:
display_data(df, selected)

Column Name : datetime, Distinct values: 10450
Column Name : hour, Distinct values: 24
Column Name : weekday, Distinct values: 7
Column Name : day, Distinct values: 19
Column Name : month, Distinct values: 12
Column Name : year, Distinct values: 2


In [9]:
df_test = pd.read_csv("bike_test.csv")
df_test.head(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,05-06-2012 5:00,2,0,1,3,19.680000,23.48500,82,11.00140
1,19-03-2011 19:00,1,0,0,1,18.040000,21.97000,33,19.00120
2,02-04-2012 6:00,2,0,1,1,13.885038,16.24664,68,29.52646
3,13-07-2012 20:00,3,0,1,1,30.340000,33.33500,48,8.99810
4,09-11-2011 19:00,4,0,1,1,18.040000,21.97000,72,7.00150


In [10]:
display_all(df_test)

Column Name : datetime, Distinct values: 2519
Column Name : season, Distinct values: 4
Column Name : holiday, Distinct values: 2
Column Name : workingday, Distinct values: 2
Column Name : weather, Distinct values: 3
Column Name : temp, Distinct values: 468
Column Name : atemp, Distinct values: 477
Column Name : humidity, Distinct values: 85
Column Name : windspeed, Distinct values: 431


In [11]:
df_test = add_features(df_test)
#selected = ["datetime", "hour", "weekday", "day", "month", "year"]
print(df_test[selected])

                datetime  hour  weekday  day  month  year
0    2012-06-05 05:00:00     5        1    5      6  2012
1    2011-03-19 19:00:00    19        5   19      3  2011
2    2012-04-02 06:00:00     6        0    2      4  2012
3    2012-07-13 20:00:00    20        4   13      7  2012
4    2011-11-09 19:00:00    19        2    9     11  2011
...                  ...   ...      ...  ...    ...   ...
2608 2012-11-07 05:00:00     5        2    7     11  2012
2609 2011-12-16 05:00:00     5        4   16     12  2011
2610 2011-01-14 23:00:00    23        4   14      1  2011
2611 2011-02-19 09:00:00     9        5   19      2  2011
2612 2012-03-17 05:00:00     5        5   17      3  2012

[2613 rows x 6 columns]


In [12]:
print(df_test.shape)

(2613, 16)


In [13]:
print(df.isna().sum())

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
hour          0
weekday       0
day           0
month         0
year          0
hour_sin      0
hour_cos      0
dtype: int64


In [14]:
#display_data(df_test, selected)

In [15]:
#read old csv
old = pd.read_csv("submission_gb_old.csv")

In [16]:
new = pd.read_csv("submission.csv")

In [17]:
new["count_old"] = old["count"]
new["difference"] = new["count"] - new["count_old"]
print(new)

              datetime  count  count_old  difference
0      05-06-2012 5:00     25         12          13
1     19-03-2011 19:00    108        103           5
2      02-04-2012 6:00     83         41          42
3     13-07-2012 20:00    384        400         -16
4     09-11-2011 19:00    286        221          65
...                ...    ...        ...         ...
2608   07-11-2012 5:00     22         30          -8
2609  16-12-2011 05:00     20         19           1
2610  14-01-2011 23:00     22         23          -1
2611  19-02-2011 09:00     66         67          -1
2612  17-03-2012 05:00      9         10          -1

[2613 rows x 4 columns]


In [18]:
y_new = new["count"]
y_old = new["count_old"]
results = {
        "RMSLE": rmsle(y_new, y_old),
        #"RMSLE-Sklearn": np.sqrt(mean_squared_log_error(y_test, y_pred)),
        #"RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        #"MAE": mean_absolute_error(y_test, y_pred),
        "R2": r2_score(y_new, y_old)
    }

# Print results
print(results)

{'RMSLE': 0.3307985725189733, 'R2': 0.9257347078559136}


In [19]:
df = pd.read_csv("submission_CB_GB.csv")
df["difference"] = df["count_CB"] - df["count_GB"]
print(df)

              datetime  count_CB  count_GB  difference
0      05-06-2012 5:00        22        37         -15
1     19-03-2011 19:00       128       149         -21
2      02-04-2012 6:00        87        87           0
3     13-07-2012 20:00       372       414         -42
4     09-11-2011 19:00       313       294          19
...                ...       ...       ...         ...
2608   07-11-2012 5:00        31        46         -15
2609  16-12-2011 05:00        22        28          -6
2610  14-01-2011 23:00        28        24           4
2611  19-02-2011 09:00        65        69          -4
2612  17-03-2012 05:00         9         7           2

[2613 rows x 4 columns]


In [20]:
df.to_csv("submission_CB_GB_compare.csv", index=False)